In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import re

In [3]:
df = pd.read_excel('PSGC 2Q 2020 Publication.xlsx', sheet_name='PSGC', dtype={'Code': object})
df.head()

,Code,Name,Geographic Level,City Class,Income\nClassification,Urban / Rural\n(based on 2015 POPCEN),POPULATION\n(2015 POPCEN),Unnamed: 7
0,010000000,REGION I (ILOCOS REGION),Reg,NaN,NaN,NaN,5026128,NaN
1,012800000,ILOCOS NORTE,Prov,NaN,1st,NaN,593081,NaN
2,012801000,ADAMS,Mun,NaN,5th,NaN,1792,NaN
3,012801001,Adams (Pob.),Bgy,NaN,NaN,R,1792,NaN
4,012802000,BACARRA,Mun,NaN,3rd,NaN,32215,NaN


In [ ]:
df = df[['Code', 'Name', 'Geographic Level', 'City Class', 'Income\nClassification']]
df.columns = ['Code', 'Name_1', 'Geographic Level', 'City Class', 'Income Classification']
df['Code_1'] = [str(i) for i in df['Code']]
df['Code_2'] = [(df['Code_1'].iloc[i][0:4] + '00000') for i in df.index]
# df['Code_1'] = [(df['Code'].iloc[i][0:4] + '00000') if len(df['Code_1'].iloc[i]) == 8 else (df['Code_1'].iloc[i][0:4] + '00000') for i in df.index]

df_prov = df[(df['Geographic Level'] == 'Prov') | (df['Geographic Level'] == 'Dist')]
df_prov = df_prov[['Code', 'Name_1']]
df_prov['Code_2'] = [str(i) for i in df_prov['Code']]
df_prov = df_prov[['Code_2', 'Name_1']]
df_prov.columns = ['Code_2', 'Name_2']

df_1 = pd.merge(df, df_prov, how='left')
df_2 = df_1[(df_1['Geographic Level'] == 'City') | (df['Geographic Level'] == 'Mun')]
df_2['Name_2'].fillna(' ', inplace = True)
df_2['Name_3'] = df_2['Name_1'] + ' ' + df_2['Name_2'] + ' ' + 'Philippines'
df_2['Income Classification'] = [str(i) for i in df_2['Income Classification']]
df_2 = df_2.reset_index(drop=True)
df_2['IC'] = [(df_2['Income Classification'].iloc[i][0:1]) for i in df_2.index]
df_2['IC'] = df_2['IC'].replace({"S":"1", "-": "6"})
df_2['IC'] = df_2['IC'].astype(int)
# df_2[df_2['Name_2'].isnull()]

df_2['Name_4'] = df_2.apply(lambda x: re.sub('\(.*?\)', '', x.Name_3).strip(), axis=1)
df_2['Name_4'] = df_2['Name_4'].str.lower()

In [ ]:
df_2.to_csv('result_2.csv')